**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
suppressMessages(suppressWarnings(library("DESeq2")))
show_env()

You are in Singularity: singularity_proj_combeffect 
BASE DIRECTORY:     /data/reddylab/Kuei 
WORK DIRECTORY:     /data/reddylab/Kuei/out 
CODE DIRECTORY:     /data/reddylab/Kuei/code 
PATH OF SOURCE:     /data/reddylab/Kuei/source 
PATH OF EXECUTABLE: /data/reddylab/Kuei/bin 
PATH OF ANNOTATION: /data/reddylab/Kuei/annotation 
PATH OF PROJECT:    /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS:    /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 


## Import count matrix and metadata

In [2]:
PREFIX = "A001_K562_WSTARRseq"
FOLDER = "coverage_astarrseq_peak_macs_input"

fdiry = file.path(FD_RES, "results", PREFIX, FOLDER, "summary")

fname = "matrix.raw.count.WGS.tsv"
fpath = file.path(fdiry, fname)
dat_count = read_tsv(fpath)

fname = "metadata.raw.WGS.tsv"
fpath = file.path(fdiry, fname)
dat_meta = read_tsv(fpath)

Rows: 246832 Columns: 11
── Column specification ──────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): Chrom, Peak
dbl (9): Start, End, Input.rep1, Input.rep2, Input.rep3, Input.rep4, Output....

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 7 Columns: 3
── Column specification ──────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): Sample, Group, FPath

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


**Arrange count matrix and metadata**

In [3]:
dat_col = dat_meta  %>% 
    dplyr::select(Sample, Group) %>% 
    dplyr::rename(condition = Group) %>%
    column_to_rownames(var = "Sample")

dat_cnt = dat_count %>% 
    dplyr::mutate(Peak = paste(Chrom, Start, End, sep = "_")) %>%
    dplyr::select(-Chrom, -Start, -End) %>%
    column_to_rownames(var = "Peak")

dat_cnt[is.na(dat_cnt)] = 0

In [4]:
head(dat_cnt)

,Input.rep1,Input.rep2,Input.rep3,Input.rep4,Output.rep1,Output.rep2,Output.rep3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1_10015_10442,1,1,1,1,0,0,0
chr1_17237_17772,5,12,15,17,18,23,27
chr1_136071_137429,3,4,7,4,12,12,29
chr1_137737_139544,14,40,41,52,145,144,217
chr1_180982_182087,8,31,26,28,63,57,99
chr1_183239_184602,12,40,36,49,71,75,163


In [5]:
dat_col

,condition
,<chr>
Input.rep1,Input
Input.rep2,Input
Input.rep3,Input
Input.rep4,Input
Output.rep1,Output
Output.rep2,Output
Output.rep3,Output


In [6]:
print(all(rownames(dat_col) %in% colnames(dat_cnt)))
print(all(rownames(dat_col) ==   colnames(dat_cnt)))

[1] TRUE
[1] TRUE


## Run DESeq2

In [7]:
dds = DESeqDataSetFromMatrix(
    countData = dat_cnt, 
    colData   = dat_col, 
    design    = ~condition)

converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


In [8]:
### remove the peaks which have < 10 reads
dds = dds[rowSums(counts(dds)) >= 10,]

### set control condition as reference
dds$condition <- relevel(dds$condition, ref = "Input")

In [9]:
dds = DESeq(dds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



## Get results

In [10]:
resultsNames(dds)

[1] "Intercept"                 "condition_Output_vs_Input"

In [11]:
res = results(dds)
res

log2 fold change (MLE): condition Output vs Input 
Wald test p-value: condition Output vs Input 
DataFrame with 246688 rows and 6 columns
                          baseMean log2FoldChange     lfcSE      stat
                         <numeric>      <numeric> <numeric> <numeric>
chr1_17237_17772           15.5078     -0.5519742  0.365031 -1.512129
chr1_136071_137429          8.0054      0.4218478  0.513401  0.821672
chr1_137737_139544         68.3888      0.7590524  0.190913  3.975910
chr1_180982_182087         35.0933      0.1919213  0.241753  0.793874
chr1_183239_184602         49.5657      0.0837488  0.201836  0.414934
...                            ...            ...       ...       ...
chr8_31049771_31050055    1.015907        2.01375   1.42070   1.41743
chr19_58604611_58605025   0.616336        2.33324   1.65053   1.41363
chr2_152740678_152740972  0.814607        2.67922   1.53733   1.74277
chr22_21396943_21397355   0.713538        2.52816   1.58085   1.59924
chr8_12223774_12224144

In [12]:
res = results(dds)
res = as.data.frame(res) %>% rownames_to_column(var = "Peak")
head(res)

,Peak,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr1_17237_17772,15.507847,-0.55197423,0.3650312,-1.5121288,0.1305011090,0.376522564
2,chr1_136071_137429,8.005396,0.42184778,0.5134014,0.8216725,0.4112633282,NA
3,chr1_137737_139544,68.388785,0.75905245,0.1909129,3.9759105,0.0000701105,0.001384044
4,chr1_180982_182087,35.093278,0.19192129,0.2417529,0.7938737,0.4272689717,0.696317984
5,chr1_183239_184602,49.565727,0.08374879,0.2018363,0.4149342,0.6781900855,0.857342193
6,chr1_186238_187159,50.450003,-0.13877450,0.2161170,-0.6421267,0.5207908905,0.763065578


## Save results

In [13]:
fdiry = file.path(FD_RES, "results", PREFIX, FOLDER, "summary")
fname = "result.Log2FC.raw.deseq.WGS.tsv"
fpath = file.path(fdiry, fname)
print(fpath)

write_tsv(res, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/coverage_astarrseq_peak_macs_input/summary/result.Log2FC.raw.deseq.WGS.tsv"
